In [ ]:
from pathlib import Path

from torch.utils.data import DataLoader

from utils import read_corpus, Vocab, collate_fn

# import torch
# from torch.nn.utils.rnn import pad_sequence
# def collate_fn(batch):
#     src_sentences, tgt_sentences = zip(*batch)
#     src_sentences = [torch.tensor(sent) for sent in src_sentences]
#     tgt_sentences = [torch.tensor(sent) for sent in tgt_sentences]
#     src_sentences = pad_sequence(src_sentences, batch_first=True)
#     tgt_sentences = pad_sequence(tgt_sentences, batch_first=True)
#     return src_sentences, tgt_sentences

%load_ext autoreload
%autoreload 2

In [ ]:
batch_size = 8
shuffle = True

data_path = Path("../code")

In [ ]:
train_data_src = read_corpus(data_path.joinpath("zh_en_data", "train_debug.zh"), data_path / "src.model")
train_data_tgt = read_corpus(data_path.joinpath("zh_en_data", "train_debug.en"), data_path / "tgt.model")

valid_data_src = read_corpus(data_path.joinpath("zh_en_data", "dev.en"), data_path / "src.model")
valid_data_tgt = read_corpus(data_path.joinpath("zh_en_data", "dev.zh"), data_path / "tgt.model")

vocab = Vocab.load(data_path / "vocab.json")
print(vocab)

train_data_indices_src = vocab.src.words2indices(train_data_src)
train_data_indices_tgt = vocab.tgt.words2indices(train_data_tgt)

valid_data_indices_src = vocab.src.words2indices(valid_data_src)
valid_data_indices_tgt = vocab.tgt.words2indices(valid_data_tgt)

In [ ]:
train_data = list(zip(train_data_indices_src, train_data_indices_tgt))
valid_data = list(zip(valid_data_indices_src, valid_data_indices_tgt))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

src_sentences, tgt_sentences, _ = next(iter(train_loader))
print(f"src shape: {src_sentences.shape}, tgt shape: {tgt_sentences.shape}")

print("src and tgt sentences:")
for src_sent, tgt_sent in zip(src_sentences, tgt_sentences):
    src_sent = "".join(vocab.src.indices2words(src_sent.tolist()))
    tgt_sent = "".join(vocab.tgt.indices2words(tgt_sent.tolist()))
    print("-" * 100)
    print(src_sent.replace("<pad>", "").replace("▁", " "))
    print(tgt_sent.replace("<pad>", "").replace("▁", " ").replace("</s>", "").replace("<s>", ""))
    print("-" * 100)
    print()